## 1. Download Precipitation and Isotope NEON data

In [48]:
# Load rpy2 so R scripts can run within jupyter notebook - run this one time
%load_ext rpy2.ipython

In [49]:
%%R 
# This rpy2 'Rmagic' command allows to run the entire cell block as R code

# Install R packages - run this one time
install.packages("neonUtilities")
install.packages('openxlsx')


--- Please select a CRAN mirror for use in this session ---
Secure CRAN mirrors 

 1: 0-Cloud [https]
 2: Australia (Canberra) [https]
 3: Australia (Melbourne 1) [https]
 4: Australia (Melbourne 2) [https]
 5: Australia (Perth) [https]
 6: Austria [https]
 7: Belgium (Ghent) [https]
 8: Brazil (BA) [https]
 9: Brazil (PR) [https]
10: Brazil (RJ) [https]
11: Brazil (SP 1) [https]
12: Brazil (SP 2) [https]
13: Bulgaria [https]
14: Canada (MB) [https]
15: Chile (Santiago) [https]
16: China (Beijing 2) [https]
17: China (Hefei) [https]
18: China (Hong Kong) [https]
19: China (Guangzhou) [https]
20: China (Lanzhou) [https]
21: China (Nanjing) [https]
22: China (Shanghai 1) [https]
23: China (Shanghai 2) [https]
24: Costa Rica [https]
25: Denmark [https]
26: East Asia [https]
27: Ecuador (Cuenca) [https]
28: Ecuador (Quito) [https]
29: Estonia [https]
30: France (Lyon 1) [https]
31: France (Lyon 2) [https]
32: France (Marseille) [https]
33: France (Montpellier) [https]
34: Germany (Erlangen

R[write to console]: also installing the dependencies ‘R.oo’, ‘R.methodsS3’, ‘R.utils’


R[write to console]: trying URL 'https://cloud.r-project.org/bin/macosx/contrib/4.0/R.oo_1.24.0.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 967806 bytes (945 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to c


The downloaded binary packages are in
	/var/folders/39/b5x92hdd1g3g_gn215gj2yxw0000gn/T//RtmpGTwDi7/downloaded_packages


R[write to console]: trying URL 'https://cloud.r-project.org/bin/macosx/contrib/4.0/openxlsx_4.2.3.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 3027965 bytes (2.9 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wri


The downloaded binary packages are in
	/var/folders/39/b5x92hdd1g3g_gn215gj2yxw0000gn/T//RtmpGTwDi7/downloaded_packages


In [50]:
%%R

# Load required libraries - run this every time
library(neonUtilities)
library(openxlsx)

# creating a list of site names: this will iterate through all sites, or can be adapted to only download sites of interest
siteNames <- c("DELA", "LENO", "TALL", "BARR", "BONA", "HEAL", "TOOL", "SRER", "SJER", "CPER", "NIWO", "RMNP",
             "STER", "OSBS", "JERC", "PUUM", "KONZ", "UKFS", "SERC", "HARV", "UNDE", "BART", "NOGP", "WOOD", "OAES",
              "GUAN", "GRSM", "ORNL", "CLBJ", "MOAB", "ONAQ", "BLAN", "MLBS", "SCBI", "WREF", "STEI", "YELL")

for (val in siteNames) {
    siteName <- val
    
    # download isotope data
    IsoData <- loadByProduct(dpID="DP1.00038.001", site=siteName)
    list2env(IsoData, .GlobalEnv)
    destination <- paste("IsoData/", siteName, "IsoData.xlsx", sep="") # moves into IsoData folder and creates file name
    write.xlsx(wdi_isoPerSample, destination, row.names=F)
    
    # download precip data - primary data if available, secondary data if not
    PrecipData <- loadByProduct(dpID="DP1.00006.001", site=siteName)
    list2env(PrecipData, .GlobalEnv)
    destination <- paste("PrecipData/", siteName, "PrecipData.xlsx", sep="") # moves into PrecipData folder and creates file name
    if (exists("PRIPRE_30min")) {
        write.xlsx(PRIPRE_30min, destination, row.names=F)
    } else {
        write.xlsx(SECPRE_30min, destination, row.names=F)
    }
}

Finding available files
  |======================================================================| 100%

Continuing will download files totaling approximately 0.245541 MB. Do you want to proceed y/n: y
  |======================================================================| 100%

Unpacking zip files using 1 cores.
  |                                                  | 0 % ~calculating   |++++++                                            | 11% ~00s           |++++++++++++                                      | 22% ~00s           |+++++++++++++++++                                 | 33% ~00s           |+++++++++++++++++++++++                           | 44% ~00s           |++++++++++++++++++++++++++++                      | 56% ~00s           |++++++++++++++++++++++++++++++++++                | 67% ~00s           |+++++++++++++++++++++++++++++++++++++++           | 78% ~00s           |+++++++++++++++++++++++++++++++++++++++++++++     | 89% ~00s           |+++++++++++++++++++++++++++++++

  |                                                  | 0 % ~calculating   |+                                                 | 1 % ~01s           |++                                                | 3 % ~01s           |+++                                               | 4 % ~01s           |+++                                               | 5 % ~01s           |++++                                              | 7 % ~01s           |+++++                                             | 8 % ~01s           |+++++                                             | 10% ~01s           |++++++                                            | 11% ~01s           |+++++++                                           | 12% ~01s           |+++++++                                           | 14% ~01s           |++++++++                                          | 15% ~01s           |+++++++++                                         | 16% ~01s           |+++++++++                                         | 18% ~01s 

  |                                                  | 0 % ~calculating   |+                                                 | 1 % ~04s           |++                                                | 3 % ~03s           |+++                                               | 4 % ~03s           |+++                                               | 5 % ~03s           |++++                                              | 7 % ~03s           |+++++                                             | 8 % ~04s           |+++++                                             | 10% ~04s           |++++++                                            | 11% ~04s           |+++++++                                           | 12% ~05s           |+++++++                                           | 14% ~05s           |++++++++                                          | 15% ~04s           |+++++++++                                         | 16% ~04s           |+++++++++                                         | 18% ~05s 

  |                                                  | 0 % ~calculating   |+                                                 | 1 % ~01s           |++                                                | 3 % ~01s           |+++                                               | 4 % ~01s           |+++                                               | 5 % ~01s           |++++                                              | 7 % ~01s           |+++++                                             | 8 % ~01s           |+++++                                             | 10% ~01s           |++++++                                            | 11% ~01s           |+++++++                                           | 12% ~01s           |+++++++                                           | 14% ~01s           |++++++++                                          | 15% ~01s           |+++++++++                                         | 16% ~01s           |+++++++++                                         | 18% ~01s 

  |                                                  | 0 % ~calculating   |+                                                 | 1 % ~03s           |++                                                | 2 % ~03s           |++                                                | 3 % ~04s           |+++                                               | 4 % ~05s           |+++                                               | 5 % ~05s           |++++                                              | 6 % ~05s           |++++                                              | 7 % ~06s           |+++++                                             | 8 % ~06s           |+++++                                             | 9 % ~06s           |++++++                                            | 10% ~06s           |++++++                                            | 11% ~06s           |+++++++                                           | 12% ~06s           |+++++++                                           | 13% ~06s 

...

  |                                                  | 0 % ~calculating   |+                                                 | 1 % ~01s           |++                                                | 2 % ~01s           |++                                                | 4 % ~01s           |+++                                               | 5 % ~01s           |+++                                               | 6 % ~01s           |++++                                              | 7 % ~01s           |+++++                                             | 8 % ~01s           |+++++                                             | 10% ~01s           |++++++                                            | 11% ~01s           |++++++                                            | 12% ~01s           |+++++++                                           | 13% ~01s           |++++++++                                          | 14% ~04s           |++++++++                                          | 15% ~04s 

  |                                                  | 0 % ~calculating   |+++++                                             | 8 % ~00s           |+++++++++                                         | 17% ~01s           |+++++++++++++                                     | 25% ~01s           |+++++++++++++++++                                 | 33% ~00s           |+++++++++++++++++++++                             | 42% ~00s           |+++++++++++++++++++++++++                         | 50% ~00s           |++++++++++++++++++++++++++++++                    | 58% ~00s           |++++++++++++++++++++++++++++++++++                | 67% ~00s           |++++++++++++++++++++++++++++++++++++++            | 75% ~00s           |++++++++++++++++++++++++++++++++++++++++++        | 83% ~00s           |++++++++++++++++++++++++++++++++++++++++++++++    | 92% ~00s           |++++++++++++++++++++++++++++++++++++++++++++++++++| 100% elapsed=00s  
Stacking table wdi_collectionIso
  |                          

  |                                                  | 0 % ~calculating   |+                                                 | 1 % ~01s           |++                                                | 3 % ~02s           |+++                                               | 4 % ~02s           |+++                                               | 6 % ~03s           |++++                                              | 7 % ~03s           |+++++                                             | 9 % ~03s           |+++++                                             | 10% ~04s           |++++++                                            | 11% ~04s           |+++++++                                           | 13% ~04s           |++++++++                                          | 14% ~04s           |++++++++                                          | 16% ~04s           |+++++++++                                         | 17% ~04s           |++++++++++                                        | 19% ~04s 

  |                                                  | 0 % ~calculating   |+                                                 | 1 % ~01s           |++                                                | 3 % ~01s           |+++                                               | 4 % ~01s           |+++                                               | 6 % ~01s           |++++                                              | 7 % ~01s           |+++++                                             | 9 % ~01s           |+++++                                             | 10% ~01s           |++++++                                            | 11% ~01s           |+++++++                                           | 13% ~01s           |++++++++                                          | 14% ~01s           |++++++++                                          | 16% ~01s           |+++++++++                                         | 17% ~01s           |++++++++++                                        | 19% ~01s 

  |                                                  | 0 % ~calculating   |++                                                | 4 % ~00s           |++++                                              | 7 % ~00s           |++++++                                            | 11% ~00s           |++++++++                                          | 14% ~00s           |+++++++++                                         | 18% ~00s           |+++++++++++                                       | 21% ~00s           |+++++++++++++                                     | 25% ~00s           |+++++++++++++++                                   | 29% ~00s           |+++++++++++++++++                                 | 32% ~00s           |++++++++++++++++++                                | 36% ~00s           |++++++++++++++++++++                              | 39% ~00s           |++++++++++++++++++++++                            | 43% ~00s           |++++++++++++++++++++++++                          | 46% ~00s 

## 2. Generate a downscaled daily data product at each NEON site

In [55]:
## Python Import Statements
import os
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
from scipy import optimize
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import datasets

ROOTDIR = os.getcwd() # Home directory
CODEDIR = ROOTDIR + '/CODE/' # Code directory
DATADIR = ROOTDIR + '/DATA/' # Data directory
OUTPUTDIR = ROOTDIR + '/OUTPUT/' # Output directory

os.chdir(CODEDIR) # Change directory
import changeTimes
import getRunningMean
import conditional_copula_ts
import iso_sine_signal
import calcSiteStats

os.chdir(ROOTDIR) # Change directory

In [52]:
# User defined variables
sitenames = ["DELA", "LENO", "TALL", "BONA", "HEAL", "TOOL", "SRER", "SJER", "CPER", "NIWO", "RMNP", "STER",
             "OSBS", "JERC", "PUUM", "KONZ", "UKFS", "SERC", "HARV", "UNDE", "BART", "NOGP", "WOOD", "OAES",
             "GUAN", "GRSM", "ORNL", "CLBJ", "MOAB", "ONAQ", "BLAN", "MLBS", "SCBI", "WREF", "STEI", "YELL"]
ensemble_size = 10

In [56]:
## This iterates through all the sites in 'sitenames'
amplitude = []
correlation = []

for s in np.arange(len(sitenames)):
    # 30min Precipitation Data
    df_P30 = pd.read_excel(DATADIR+'PrecipData/'+str(sitenames[s])+'PrecipData.xlsx',index=False)
    df_P30 = changeTimes.change_Pdata(df_P30)
    
    # Sum to Daily Precipitation Amounts
    # checks for primary precip data
    if 'priPrecipBulk' in df_P30:
        df_P30b = df_P30.set_index('DateTime')
        precip_daily = df_P30b['priPrecipBulk'].resample('D').sum() # sum total daily P
        frac_year = df_P30b['FracYear'].resample('D').mean() # average daily FracYear

        daily_P = pd.DataFrame({'Total P':precip_daily,'FracYear':frac_year})
        daily_P['Total P'].replace(0, np.nan, inplace=True)


        # Biweekly Stable Water Isotope Data
        df_iso = pd.read_excel(DATADIR+'IsoData/'+str(sitenames[s])+'IsoData.xlsx',index=False)
        df_iso = changeTimes.change_ISOdata(df_iso)



        # Create Biweekly Precipitation Amount Timeseries to Correspond to Recorded Isotope Values
        df_iso['setDate'] = pd.to_datetime(df_iso['setDate'])  
        df_iso['collectDate'] = pd.to_datetime(df_iso['collectDate']) 

        P14 = []
        for i in np.arange(len(df_iso['setDate'])):
            subset = ((df_P30['DateTime'] > df_iso['setDate'].iloc[i]) 
                    & (df_P30['DateTime'] <= df_iso['collectDate'].iloc[i]))

            df_sub = df_P30.loc[subset]
            P14.append(np.nansum(df_sub['priPrecipBulk'].values))

        df_iso['Total P'] = P14
        del P14, i, subset
    
    # uses secondary precip data
    else:
        df_P30b = df_P30.set_index('DateTime')
        precip_daily = df_P30b['secPrecipBulk'].resample('D').sum() # sum total daily P
        frac_year = df_P30b['FracYear'].resample('D').mean() # average daily FracYear

        daily_P = pd.DataFrame({'Total P':precip_daily,'FracYear':frac_year})
        daily_P['Total P'].replace(0, np.nan, inplace=True)


        # Biweekly Stable Water Isotope Data
        df_iso = pd.read_excel(DATADIR+'IsoData/'+str(sitenames[s])+'IsoData.xlsx',index=False)
        df_iso = changeTimes.change_ISOdata(df_iso)



        # Create Biweekly Precipitation Amount Timeseries to Correspond to Recorded Isotope Values
        df_iso['setDate'] = pd.to_datetime(df_iso['setDate'])  
        df_iso['collectDate'] = pd.to_datetime(df_iso['collectDate']) 

        P14 = []
        for i in np.arange(len(df_iso['setDate'])):
            subset = ((df_P30['DateTime'] > df_iso['setDate'].iloc[i]) 
                    & (df_P30['DateTime'] <= df_iso['collectDate'].iloc[i]))

            df_sub = df_P30.loc[subset]
            P14.append(np.nansum(df_sub['secPrecipBulk'].values))

        df_iso['Total P'] = P14
        del P14, i, subset
    
    
    '''
    Step 1 - Remove seasonal time series component (Section 2.b.1)
    '''
    
    # Define Sine Wave Functions
    df_iso = df_iso.sort_values('DateTime')
    df_iso = df_iso.dropna(subset=['Total P'])
    tsX = df_iso['FracYear'].values 
    tsP = df_iso['Total P'].values
    tsO = df_iso['d18OWater'].values
    tsH = df_iso['d2HWater'].values   

    dayslist = []
    for dt in np.arange(len(df_iso['DateTime'])):
        dayslist.append((df_iso['DateTime'].iloc[dt] - df_iso['DateTime'].iloc[0]).days)
    dayslist = np.array(dayslist)

    # lambda is precipitation frequency (see Eq. 4)
    p_events = df_iso[df_iso['Total P'].notna()]
    lamda = len(p_events['Total P'])/((daily_P.index.max() - daily_P.index.min()).days) 

    params1, params2 = iso_sine_signal.sine_params(sitenames[s],14,tsX,tsP,tsH,tsO) # 14 = biweekly sample, sample frequency
    tsY_sine_wave = iso_sine_signal.sine_func(tsX, params1[0],params1[1],params1[2]) # solve for amplitude, phase, offset
    adj_2H = np.array((tsH - tsY_sine_wave))    # remove seasonality from time series                       

    tsY_sine_wave = iso_sine_signal.sine_func(tsX, params2[0],params2[1],params2[2])     
    adj_18O = np.array((tsO - tsY_sine_wave)) 
    
    amplitude.append(params1[0])
    
    # Get biweekly site stats of stochastic component
    biweekly_stats = np.array([[np.mean(tsP), np.mean(adj_2H), np.mean(adj_18O)],
                     [np.std(tsP), np.std(adj_2H), np.std(adj_18O)],
                     [sp.stats.pearsonr(tsP,adj_2H)[0], sp.stats.pearsonr(tsP,adj_18O)[0], sp.stats.pearsonr(adj_2H,adj_18O)[0]]])

    correlation.append(sp.stats.pearsonr(tsP,adj_18O)[0])
    
    '''
    Step 2 - Predict daily statistics from biweekly time series (Section 2.b.2) Now we will need to aggregate the 
    stochastic biweekly time series - i.e. calculated weighed running means at biweekly (14-day), 28-day, 42-day, 
    56-day and 84-day intervals    
    '''

    # function defining calculated statistics on time series
    site_stats = [[sitenames[s], 14, lamda,
                   biweekly_stats[0,0], biweekly_stats[0,1], biweekly_stats[0,2],
                   biweekly_stats[1,0], biweekly_stats[1,1], biweekly_stats[1,2],
                   biweekly_stats[2,0], biweekly_stats[2,1], biweekly_stats[2,2]]]
    site_stats_check = len(site_stats)

    for n in np.arange(28,85,14):
        xday_Hb, xday_Pb, xday_Xb, days = getRunningMean.main(np.array(dayslist),np.array(tsH),np.array(tsP),tsX,n)
        xday_Ob, xday_Pb, xday_Xb, days = getRunningMean.main(np.array(dayslist),np.array(tsO),np.array(tsP),tsX,n)  

        xday_Xb = np.array(xday_Xb)
        xday_Pb = np.array(xday_Pb)
        xday_Hb = np.array(xday_Hb)
        xday_Ob = np.array(xday_Ob)   

        params1a, params2a = iso_sine_signal.sine_params(sitenames[s],n,xday_Xb,xday_Pb,xday_Hb,xday_Ob)

        tsY_sine_wave = iso_sine_signal.sine_func(xday_Xb, params1a[0],params1a[1],params1a[2])
        adj_2Hb = np.array((xday_Hb - tsY_sine_wave))                           

        tsY_sine_wave = iso_sine_signal.sine_func(xday_Xb, params2a[0],params2a[1],params2a[2])     
        adj_18Ob = np.array((xday_Ob - tsY_sine_wave))
        
        if len(xday_Pb)>2 and len(adj_2Hb)>2 and len(adj_18Ob)>2:
            calcSiteStats.main(site_stats,sitenames[s],lamda,n,xday_Pb,adj_2Hb,adj_18Ob)
            
    if len(site_stats)==site_stats_check:
        print("Site {} does not contain sufficient data.".format(sitenames[s]))
        
    else:
        ### the stats are labeled with 'B' here because they are of the stochastic component - not the original time series
        Site_Stats = pd.DataFrame(site_stats, columns = ['site','agglev','lambda','PmuB','HmuB','OmuB','PsigB',
                                                         'HsigB','OsigB','PHpB','POpB','HOpB'])
    
        ### Now we apply equation 4
        Hi = [] ; Oi = [] # estimated 1-day parameters
        H_a = [] ; O_a = [] # a term

        xaxis = np.array(Site_Stats['agglev'].values)
        yaxis1 = np.array(Site_Stats['HsigB'].values)
        yaxis2 = np.array(Site_Stats['OsigB'].values)

        def eq4(x,a,b):
            return b/(x*lamda)**a

        bounds = [[0.2,yaxis1[0]],[0.5,np.inf]]
        p1,p2 = optimize.curve_fit(eq4, xaxis, yaxis1, p0 = [0.3,yaxis1[0]], bounds=bounds)

        Hi.append(float(p1[1]))
        H_a.append(p1[0])

        bounds = [[0.2,yaxis2[0]],[0.5,np.inf]]
        p1,p2 = optimize.curve_fit(eq4, xaxis, yaxis2,  p0 = [0.3,yaxis2[0]], bounds=bounds)

        Oi.append(float(p1[1]))
        O_a.append(p1[0])
    
        
        '''
        Step 3 - Generate daily time series with estimated statistcs (Section 2.b.2)
        Step 4 - Add in seasonal time series component (Section 2.b.2)
        '''
    
        H_scale = np.sort(np.array(adj_2H) * Hi / Site_Stats['HsigB'].iloc[0])
        O_scale = np.sort(np.array(adj_18O) * Oi / Site_Stats['OsigB'].iloc[0])

        copula_stats = np.matrix([[0, 0, 0], [np.std(tsP), Hi, Oi],
                                    [Site_Stats['PHpB'].iloc[0], 
                                     Site_Stats['POpB'].iloc[0],
                                     Site_Stats['HOpB'].iloc[0]]])    

        for num in np.arange(1,ensemble_size+1):
            new_ts = conditional_copula_ts.main(daily_P['Total P'],copula_stats,H_scale,O_scale)
            y = np.array([np.array(xi) for xi in new_ts])   

            # Add back in n-day sine function here:
            tsH_daily = y[:,0] + iso_sine_signal.sine_func(daily_P['FracYear'], params1[0],params1[1],params1[2])    
            tsO_daily = y[:,1] + iso_sine_signal.sine_func(daily_P['FracYear'], params2[0],params2[1],params2[2])
        
            columnA = 'd2H_'+str(num)
            columnB = 'd18O_'+str(num)
            daily_P[columnA] = tsH_daily
            daily_P[columnB] = tsO_daily 

        daily_P.to_csv(ROOTDIR+'/OUTPUT/'+sitenames[s]+'_daily_timeseries.csv')
%store amplitude
%store correlation

/Users/lindseyspencer/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:799: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/lindseyspencer/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:799: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/lindseyspencer/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/Users/lindseyspencer/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:799: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/lindseyspencer/anaconda3/lib/python3.7/site-packages/scipy/optimize/minpack.py:799: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/linds

Stored 'amplitude' (list)
Stored 'correlation' (list)


## 3. Residual Correction to Make Statistics Match

In [57]:
    
for s in np.arange(len(sitenames)):
    sitename = sitenames[s]
    DateTime = []
    Total_P = []
    FracYear = []
    corrected_d2H = []
    corrected_d18O = []
    
    # creating biweekly precip amounts (same code as above)
    df_P30 = pd.read_excel(DATADIR+'PrecipData/'+str(sitenames[s])+'PrecipData.xlsx',index=False)
    df_P30 = changeTimes.change_Pdata(df_P30)
    
    # checks for primary precip data
    if 'priPrecipBulk' in df_P30:
        df_P30b = df_P30.set_index('DateTime')
        precip_daily = df_P30b['priPrecipBulk'].resample('D').sum() # sum total daily P
        frac_year = df_P30b['FracYear'].resample('D').mean() # average daily FracYear

        daily_P = pd.DataFrame({'Total P':precip_daily,'FracYear':frac_year})
        daily_P['Total P'].replace(0, np.nan, inplace=True)


        # Biweekly Stable Water Isotope Data
        df_iso = pd.read_excel(DATADIR+'IsoData/'+str(sitenames[s])+'IsoData.xlsx',index=False)
        df_iso = changeTimes.change_ISOdata(df_iso)



        # Create Biweekly Precipitation Amount Timeseries to Correspond to Recorded Isotope Values
        df_iso['setDate'] = pd.to_datetime(df_iso['setDate'])  
        df_iso['collectDate'] = pd.to_datetime(df_iso['collectDate']) 

        P14 = []
        for i in np.arange(len(df_iso['setDate'])):
            subset = ((df_P30['DateTime'] > df_iso['setDate'].iloc[i]) 
                    & (df_P30['DateTime'] <= df_iso['collectDate'].iloc[i]))

            df_sub = df_P30.loc[subset]
            P14.append(np.nansum(df_sub['priPrecipBulk'].values))

        df_iso['Total P'] = P14
        del P14, i, subset
    
    # uses secondary precip data
    else:
        df_P30b = df_P30.set_index('DateTime')
        precip_daily = df_P30b['secPrecipBulk'].resample('D').sum() # sum total daily P
        frac_year = df_P30b['FracYear'].resample('D').mean() # average daily FracYear

        daily_P = pd.DataFrame({'Total P':precip_daily,'FracYear':frac_year})
        daily_P['Total P'].replace(0, np.nan, inplace=True)


        # Biweekly Stable Water Isotope Data
        df_iso = pd.read_excel(DATADIR+'IsoData/'+str(sitenames[s])+'IsoData.xlsx',index=False)
        df_iso = changeTimes.change_ISOdata(df_iso)



        # Create Biweekly Precipitation Amount Timeseries to Correspond to Recorded Isotope Values
        df_iso['setDate'] = pd.to_datetime(df_iso['setDate'])  
        df_iso['collectDate'] = pd.to_datetime(df_iso['collectDate']) 

        P14 = []
        for i in np.arange(len(df_iso['setDate'])):
            subset = ((df_P30['DateTime'] > df_iso['setDate'].iloc[i]) 
                    & (df_P30['DateTime'] <= df_iso['collectDate'].iloc[i]))

            df_sub = df_P30.loc[subset]
            P14.append(np.nansum(df_sub['secPrecipBulk'].values))

        df_iso['Total P'] = P14
        del P14, i, subset
    
    # goes through each biweekly period in the isotope data
    for n in np.arange(len(df_iso['setDate'])):
        start_date = df_iso['setDate'].iloc[n]
        end_date = df_iso['collectDate'].iloc[n]
        
        # calculate observed H and O
        obs_Hn = df_iso['d2HWater'].iloc[n]
        obs_On = df_iso['d18OWater'].iloc[n]
        obs_Pn = df_iso['Total P'].iloc[n]
        
        obs_d2Hw = (np.nansum(obs_Hn*obs_Pn)/np.nansum(obs_Pn))
        obs_d18Ow = (np.nansum(obs_On*obs_Pn)/np.nansum(obs_Pn))
        
        # find corresponding data in synthetic time series
        df_synthetic = pd.read_csv(OUTPUTDIR+str(sitename)+'_daily_timeseries.csv')
        df_synthetic['DateTime'] = pd.to_datetime(df_synthetic['DateTime'])
        mask = (df_synthetic['DateTime'] >= start_date) & (df_synthetic['DateTime'] <= end_date)
        df_synthetic = df_synthetic.loc[mask]
        
        # calculate synthetic H and O
        syn_Hn = np.sum(df_synthetic['d2H_1'])
        syn_On = np.sum(df_synthetic['d18O_1'])
        
        syn_d2Hw = (np.nansum(syn_Hn*obs_Pn)/np.nansum(obs_Pn))
        syn_d18Ow = (np.nansum(syn_On*obs_Pn)/np.nansum(obs_Pn))
        
        # find difference between observed and synthetic
        diff_d2Hw = obs_d2Hw - syn_d2Hw
        diff_d18Ow = obs_d18Ow - syn_d18Ow
        
        # number of days of precip
        precip_days = np.nansum(df_synthetic['Total P'].values > 0)
        
        # divide difference by days of precip to get daily difference
        diff_daily_H = diff_d2Hw / precip_days
        diff_daily_O = diff_d18Ow / precip_days

        # add difference to synthetic values
        for i in np.arange(len(df_synthetic['DateTime'])):
            DateTime.append(df_synthetic['DateTime'].iloc[i])
            Total_P.append(df_synthetic['Total P'].iloc[i])
            FracYear.append(df_synthetic['FracYear'].iloc[i])
            
            if df_synthetic['Total P'].iloc[i] > 0:
                corrected_H = df_synthetic['d2H_1'].iloc[i] + diff_daily_H
                corrected_d2H.append(corrected_H)
                corrected_O = df_synthetic['d18O_1'].iloc[i] + diff_daily_O
                corrected_d18O.append(corrected_O)
                
            else:
                corrected_d2H.append(np.nan)
                corrected_d18O.append(np.nan)
                
    # create new CSV with corrected values
    df_corrected = []
    df_corrected.append(DateTime)
    df_corrected.append(Total_P)
    df_corrected.append(FracYear)
    df_corrected.append(corrected_d2H)
    df_corrected.append(corrected_d18O)
    
    df_corrected = pd.DataFrame(df_corrected)
    df_corrected = df_corrected.T
    df_corrected.columns = ['DateTime', 'Total P', 'FracYear', 'd2H', 'd18O']
    
    df_corrected.to_csv(ROOTDIR+'/OUTPUT/'+sitenames[s]+'_corrected_daily.csv', index=False)

/Users/lindseyspencer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:106: RuntimeWarning: invalid value encountered in greater
/Users/lindseyspencer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in double_scalars
/Users/lindseyspencer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in double_scalars
/Users/lindseyspencer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:98: RuntimeWarning: invalid value encountered in double_scalars
/Users/lindseyspencer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:99: RuntimeWarning: invalid value encountered in double_scalars
/Users/lindseyspencer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:109: RuntimeWarning: divide by zero encountered in double_scalars
/Users/lindseyspencer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:110: RuntimeWarning: divide by zero encountere